In [4]:
documents = [
    { 'type': 'passport', 'number': '2207 876234', 'name': 'Василий Губкин' },
    { 'type': 'invoice', 'number': '11-2', 'name': 'Геннадий Покемонов' },
    { 'type': 'insurance', 'number': '10006', 'name': 'Аристарх Павлов' }
]
directories = {
    '1': ['2207 876234', '11-2'],
    '2': ['10006'],
    '3': []
}

In [9]:
import re
from pprint import pprint

def user_io(message_code, message_type='out', **format_args):
    """ Метод, работающий на ввод и вывод сообщений """
    
    messages = {
        'start': """
    Добро пожаловать в хранилище.
    Доступные команды:
        1. p - найти владельца документа по его номеру;
        2. s - найти полку, на которой хранится документ по его номеру;
        3. q - Завершить работу с хранилищем.""",
        
        'wrong_command': """
    Такой команды не существует""",
        
        'select_command': """
    Выберите команду: """,
        
        'enter_doc_id': """
    Введите номер документа: """,
        
        'wrong_doc_id': """
    Требуется ввести число""",
        
        'owner_found': """
    Владелец документа: {}""".format(format_args.get('name', 'Владелец не найден')),
        
        'shelf_found': f"""
    Документ хранится на полке: {format_args['shelf']}""" if 'shelf' in format_args.keys() else """
    Документ не найден""",
        
        'finish': """
    Было приятно поработать. Возвращайтесь! """
    }
    
    message = messages.get(message_code, '')
    return print(message) if message_type == 'out' else input(message)

def parse_doc_id(doc_id):
    """ Парсим номер документа в валидное целочисленное значение """
    
    return int(re.sub('\s|-', '', doc_id))

def get_doc_id():
    """ Получаем номер документа пользователя """
    
    try:
        return parse_doc_id(user_io('enter_doc_id', message_type='in'))
    except ValueError:
        return user_io('wrong_doc_id')
    

def find_owner_by_doc_id(storage):
    """ Находим владельца по номеру документа """
    
    input_doc_id = get_doc_id()
    if input_doc_id:
        for item in storage.get('documents', []):
            if parse_doc_id(item.get('number', '-1')) == input_doc_id:
                user_io('owner_found', 'out', name=item.get('name'))
                break
        else:
            user_io('owner_found')
    
    return True

def find_shelf_by_doc_id(storage):
    """ Находим полку, на которой находится документ по его номеру """
    
    input_doc_id = get_doc_id()
    if input_doc_id:
        for shelf, doc_ids in storage.get('directories', {}).items():
            parsed_doc_ids = list(map(parse_doc_id, doc_ids))

            if input_doc_id in parsed_doc_ids:
                user_io('shelf_found', 'out', shelf=shelf)
                break
        else:
            user_io('shelf_found')
    
    return True

def handle_user_command(command_id, storage_args):
    """ Метод-маршрутизатор для запуска команды по введеному коду """
    
    commands_map = {
        'p': find_owner_by_doc_id,
        's': find_shelf_by_doc_id,
        'q': lambda storage: user_io('finish')
    }
    
    return commands_map.get(
        command_id,
        lambda storage: user_io('wrong_command') or True
    )(storage_args)

def main(storage_args):
    """ Основной метод работы с хранилищем """
    user_io('start')
    command_result = True
    
    while command_result:
        command_result = handle_user_command(
            user_io('select_command', message_type='in'),
            storage_args
        )

In [10]:
main({
    'documents': documents,
    'directories': directories
})


    Добро пожаловать в хранилище.
    Доступные команды:
        1. p - найти владельца документа по его номеру;
        2. s - найти полку, на которой хранится документ по его номеру;
        3. q - Завершить работу с хранилищем.

    Выберите команду: s

    Введите номер документа: 10006

    Документ хранится на полке: 2

    Выберите команду: s

    Введите номер документа: 2207 876234

    Документ хранится на полке: 1

    Выберите команду: s

    Введите номер документа: 11-2

    Документ хранится на полке: 1

    Выберите команду: p

    Введите номер документа: asdf

    Требуется ввести число

    Выберите команду: s

    Введите номер документа: asdf

    Требуется ввести число

    Выберите команду: p

    Введите номер документа: 10006

    Владелец документа: Аристарх Павлов

    Выберите команду: p

    Введите номер документа: 234234

    Владелец документа: Владелец не найден

    Выберите команду: s

    Введите номер документа: 234234

    Документ не найден

    